In [1]:
import torch
import pandas as pd
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)

2025-06-18 23:59:49.220647: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-18 23:59:49.220724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-18 23:59:49.222375: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-18 23:59:49.233565: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-18 23:59:51.225398: W tensorflow/compiler/tf2

In [2]:
model_id = "roberta-base"

In [4]:
train_dataset = pd.read_csv('../data/certainty_train.csv', index_col=0)
train_dataset.label = pd.Categorical(train_dataset.label)
train_dataset["labels"] = 0
train_dataset.loc[train_dataset["label"] == "Certain", "labels"] = 0
train_dataset.loc[train_dataset["label"] == "Somewhat certain", "labels"] = 1
train_dataset.loc[train_dataset["label"] == "Somewhat uncertain", "labels"] = 2
train_dataset.loc[train_dataset["label"] == "Uncertain", "labels"] = 3
train_dataset = train_dataset.drop(columns=['label'])

test_dataset = pd.read_csv('../data/certainty_test.csv', index_col=0)
test_dataset.label = pd.Categorical(test_dataset.label)
test_dataset["labels"] = 0
test_dataset.loc[test_dataset["label"] == "Certain", "labels"] = 0
test_dataset.loc[test_dataset["label"] == "Somewhat certain", "labels"] = 1
test_dataset.loc[test_dataset["label"] == "Somewhat uncertain", "labels"] = 2
test_dataset.loc[test_dataset["label"] == "Uncertain", "labels"] = 3
test_dataset = test_dataset.drop(columns=['label'])

In [5]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

In [6]:
# Preprocessing
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

# This function tokenizes the input text using the RoBERTa tokenizer.
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    return tokenizer(batch["finding"], padding=True, truncation=True, max_length=1536)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [7]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Map:   0%|          | 0/1340 [00:00<?, ? examples/s]

Map:   0%|          | 0/334 [00:00<?, ? examples/s]

In [8]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [9]:
# # Create an id2label mapping
id2label = {0: "Certain", 1: "Somewhat certain", 2: "Somewhat uncertain", 3: "Uncertain"}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [10]:
# Model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# TrainingArguments
training_args = TrainingArguments(
    output_dir="../out",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir=f"../logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=200,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Fine-tune the model
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.348800
20,1.374900
30,1.351000
40,1.344700
50,1.309000
60,1.275800
70,1.264700
80,1.222600
90,1.186200
100,1.123400


TrainOutput(global_step=840, training_loss=0.9259893110820225, metrics={'train_runtime': 420.6802, 'train_samples_per_second': 15.927, 'train_steps_per_second': 1.997, 'total_flos': 805689296942400.0, 'train_loss': 0.9259893110820225, 'epoch': 5.0})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f7c41bfb350, execution_count=11 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7c41afb450, raw_cell="# Fine-tune the model
trainer.train()" store_history=True silent=False shell_futures=True cell_id=f01ca77e-fb7c-4f42-86ec-74977baf5d67> result=TrainOutput(global_step=840, training_loss=0.9259893110820225, metrics={'train_runtime': 420.6802, 'train_samples_per_second': 15.927, 'train_steps_per_second': 1.997, 'total_flos': 805689296942400.0, 'train_loss': 0.9259893110820225, 'epoch': 5.0})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [12]:
trainer.save_model("../certainty_model")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f7da8494610, raw_cell="trainer.save_model("../certainty_model")" store_history=True silent=False shell_futures=True cell_id=9371c2d0-8be7-4549-a0be-e458767ff950>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f7c1812f610, execution_count=12 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7da8494610, raw_cell="trainer.save_model("../certainty_model")" store_history=True silent=False shell_futures=True cell_id=9371c2d0-8be7-4549-a0be-e458767ff950> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [13]:
pred = trainer.predict(test_dataset)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f7c181bd010, raw_cell="pred = trainer.predict(test_dataset)" store_history=True silent=False shell_futures=True cell_id=c193ebf5-d3b6-4db5-8ea3-13c9f6ac7934>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f7c41c65c90, execution_count=13 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7c181bd010, raw_cell="pred = trainer.predict(test_dataset)" store_history=True silent=False shell_futures=True cell_id=c193ebf5-d3b6-4db5-8ea3-13c9f6ac7934> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [14]:
y_pred = pred.predictions.argmax(axis=1)
from sklearn.metrics import f1_score
f1_score(list(test_dataset['labels']), list(y_pred), average='macro')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f7c1812dc90, raw_cell="y_pred = pred.predictions.argmax(axis=1)
from skle.." store_history=True silent=False shell_futures=True cell_id=171c5bff-127d-4db3-816d-46172c682ec2>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

0.40918129489558064

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f7c18163950, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7c1812dc90, raw_cell="y_pred = pred.predictions.argmax(axis=1)
from skle.." store_history=True silent=False shell_futures=True cell_id=171c5bff-127d-4db3-816d-46172c682ec2> result=0.40918129489558064>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [15]:
f1_score(test_dataset['labels'], pred.predictions.argmax(axis=1), average=None)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f7c1812ddd0, raw_cell="f1_score(test_dataset['labels'], pred.predictions..." store_history=True silent=False shell_futures=True cell_id=869fdb61-39f1-417d-a401-9f4f23a13d61>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

array([0.68707483, 0.47272727, 0.47692308, 0.        ])

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f7c181b02d0, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7c1812ddd0, raw_cell="f1_score(test_dataset['labels'], pred.predictions..." store_history=True silent=False shell_futures=True cell_id=869fdb61-39f1-417d-a401-9f4f23a13d61> result=array([0.68707483, 0.47272727, 0.47692308, 0.        ])>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [16]:
model.save_pretrained('certainty_model')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f7c18190910, raw_cell="model.save_pretrained('certainty_model')" store_history=True silent=False shell_futures=True cell_id=5b9fce05-38bd-4216-bb5e-08fe43dba2e6>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7c41c66c10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f7c18136650, execution_count=16 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7c18190910, raw_cell="model.save_pretrained('certainty_model')" store_history=True silent=False shell_futures=True cell_id=5b9fce05-38bd-4216-bb5e-08fe43dba2e6> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given